In [1]:
import gymnasium as gym
from gymnasium import envs

# List all registered environments (including Atari)
all_envs = envs.registry.keys()
atari_envs = [env_id for env_id in all_envs if "ALE/" in env_id]

# Print first 10 Atari games
print(list(atari_envs)[:10])  # Should include 'ALE/Frogger-v5'

[]


In [2]:
import gymnasium as gym
import os
import numpy as np
import pandas as pd
import seaborn as sns
import ale_py

C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import gymnasium as gym

# Modern way to create Atari environment
env = gym.make("ALE/Frogger-v5", render_mode="human")  # Note: ALE/ prefix required


observation, info = env.reset()
for _ in range(100):
    action = env.action_space.sample()  # Random action
    observation, reward, terminated, truncated, info = env.step(action)
        
    if terminated or truncated:
        observation, info = env.reset()

In [4]:
env.close()